

### Mount Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Check GPU

In [ ]:
!nvidia-smi

Sat Nov  2 15:08:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Dataset from Huggingface

In [ ]:
cd /content/drive/MyDrive/Fine-Tune/openthaigpt-finetune-main

/content/drive/MyDrive/Fine-Tune/openthaigpt-finetune-main


In [ ]:
!dir

alpaca_data_cleaned_archive.json  docker-compose.yml		   finetune.py	   README.md
alpaca_data_gpt4.json		  Dockerfile			   generate.py	   requirements.txt
alpaca_data.json		  export_hf_checkpoint.py	   LICENSE	   templates
DATA_LICENSE			  export_state_dict_checkpoint.py  pyproject.toml  utils


### Clone Fine Tuning Code

In [ ]:
!pip install -r requirements.txt

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-m7i_j1rz
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-m7i_j1rz
  Resolved https://github.com/huggingface/peft.git to commit b5b902368d2a7f349783a96d9d3c686ea0ca3a05
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### Finetune

In [ ]:
from datasets import load_dataset

data_path = '/content/drive/MyDrive/datasets/'
dataset = load_dataset('json', data_files={'train': data_path + 'Dataset_com_Laws.json'})  # Adjust according to your dataset type and path

print(dataset['train'][0])

Generating train split: 0 examples [00:00, ? examples/s]

{'instruction': 'อธิบายความผิดเกี่ยวกับคอมพิวเตอร์', 'input': 'สวัสดีครับ ถ้าไปแอบเอาข้อมูลของคนอื่นมาแล้วไปเผยแพร่ จะมีคววามผิดอะไรไหม', 'output': 'มีความผิดมาตรา 5 ผู้ใดทำให้เสียหาย ทำลาย แก้ไข เปลี่ยนแปลง หรือเพิ่มเติมไม่ว่าทั้งหมดหรือบางส่วน ซึ่งข้อมูลคอมพิวเตอร์ของผู้อื่นโดยมิชอบ ต้องระวางโทษจำคุกไม่เกิน 5 ปี หรือปรับไม่เกิน 10,000บาท หรือทั้งจำทั้งปรับ'}


In [ ]:
!python finetune.py \
    --base_model '/content/drive/MyDrive/BaseModel/openthaigpt-1.0.0-7b-chat' \
    --data_path '/content/drive/MyDrive/datasets/Dataset_com_Laws.json' \
    --output_dir '/content/drive/MyDrive/TrainModel/Com_Laws' \
    --batch_size 128 \
    --micro_batch_size 4 \
    --num_epochs 3\
    --learning_rate 1e-4 \
    --cutoff_len 512 \
    --val_set_size 10 \
    --lora_r 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target_modules '[q_proj,v_proj]' \
    --train_on_inputs \
    --group_by_length


Training Llama v2 model with params:
base_model: /content/drive/MyDrive/BaseModel/openthaigpt-1.0.0-7b-chat
data_path: /content/drive/MyDrive/datasets/Dataset_com_Laws.json
output_dir: /content/drive/MyDrive/TrainModel/Com_Laws
batch_size: 128
micro_batch_size: 4
num_epochs: 3
learning_rate: 0.0001
cutoff_len: 512
val_set_size: 10
lora_r: 8
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules: ['q_proj', 'v_proj']
train_on_inputs: True
add_eos_token: False
group_by_length: True
wandb_project: 
wandb_run_name: 
wandb_watch: 
wandb_log_model: 
resume_from_checkpoint: False
prompt template: llama_v2

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100% 3/3 [01:20<00:00, 26.67s/it]
trainable params: 4,194,304 || all params: 6,817,058,816 || trainable%: 0.0615
Map: 100% 81/81 [00:00<00:00, 514.64 examples/s]
Map: 100% 10